In [6]:
import numpy as np 
import pandas as pd
import re 
from ast import literal_eval

In [315]:
deck = pd.read_csv('parser_output/WLL.csv')

In [316]:
len(literal_eval(deck[deck["name"] == "Arid Mesa"]["color_identity"].iloc[0]))

0

In [317]:
def find_identity(deck):
    commanders = deck[deck["board"]=="commander"]
    identity = []
    for i in range(len(commanders)):
        identity += literal_eval(commanders['color_identity'].iloc[i])
    return list(np.unique(identity))

In [318]:
deck_identity = find_identity(deck)

In [319]:
deck_identity

['B', 'R', 'U', 'W']

In [320]:
cards = deck.groupby("card_id")

In [321]:
len(cards)

101

In [322]:
np.unique(["None","None"])[0] == 'None'

True

In [333]:
card_ids = []
mana = []
for i in range(len(cards)):
    temp_identity = []
    group = cards.get_group(i)
    not_land = True
    for i in range(len(group)):
        temp_name += group.iloc[i]["name"]
        if "Land" in literal_eval(group.iloc[i]["type"]):
            not_land = False
            c_identity = literal_eval(group.iloc[i]["color_identity"])
            if len(c_identity) > 0:
                for c in c_identity:
                    temp_identity.append(c)
            else:
                color = determine_source(group.iloc[i]["text"], deck_identity)
                if color == "Undefined":
                    temp_identity.append(color)
                else:
                    for c in color:
                        temp_identity.append(c)
            

    if not_land:
        temp_identity.append("Not a mana source")
    card_ids.append(group.iloc[0]["card_id"])
    mana.append(''.join(list(np.unique(temp_identity))))

In [334]:
deck["is_dfc"].iloc[0] == False

True

In [335]:
mana_sources = pd.DataFrame({"mana":mana, "card_id":card_ids})

In [336]:
mana_sources[0:50]

,mana,card_id
0,Not a mana source,0
1,Not a mana source,1
2,Not a mana source,2
3,Not a mana source,3
4,Not a mana source,4
5,Not a mana source,5
6,Not a mana source,6
7,Not a mana source,7
8,Not a mana source,8
9,BW,9


In [337]:
len(mana)

101

In [338]:
deck_with_mana_sources = deck.merge(mana_sources, on="card_id")

In [339]:
undefined_sources = deck_with_mana_sources[deck_with_mana_sources["mana"] == "Undefined"]

In [340]:
undefined_sources

,name,cmc,text,mana_cost,card_id,is_dfc,color_identity,board,power,toughness,supertype,type,subtype,mana


In [342]:
deck_with_mana_sources[deck_with_mana_sources["name"]=="Gitaxian Probe"]

,name,cmc,text,mana_cost,card_id,is_dfc,color_identity,board,power,toughness,supertype,type,subtype,mana
52,Gitaxian Probe,1.0,({U/P} can be paid with either {U} or 2 life.)...,{U/P},52,False,['U'],mainboard,None,None,[],['Sorcery'],[],Not a mana source


In [223]:
undefined_sources[undefined_sources["name"]=="Fabled Passage"].iloc[0]["text"]

'{T}, Sacrifice Fabled Passage: Search your library for a basic land card, put it onto the battlefield tapped, then shuffle. Then if you control four or more lands, untap that land.'

In [331]:
fetch_regex = r"^{T}, Pay 1 life, Sacrifice .*: Search your library for a.* card, put it onto the battlefield, then shuffle.$"
wubrg_regex = r"Add one mana of any color"
urborg_regex = r"Each land is a Swamp in addition"
slow_fetch_regex = r"Search your library for a basic land card, put it onto the battlefield tapped"
colorless_regex = r"{T}: Add {C}"
pool_regex = r"{T}: Add one mana of any type"

In [299]:
match = re.findall(wubrg_regex, wubrg_text)

In [300]:
match

['Add one mana of any color']

In [301]:
urborg_text =  undefined_sources[undefined_sources["name"]=="Urza's Saga"].iloc[0]["text"]

In [302]:
urborg_text

'(As this Saga enters and after your draw step, add a lore counter. Sacrifice after III.)\nI — Urza\'s Saga gains "{T}: Add {C}."\nII — Urza\'s Saga gains "{2}, {T}: Create a 0/0 colorless Construct artifact creature token with \'This creature gets +1/+1 for each artifact you control.\'"\nIII — Search your library for an artifact card with mana cost {0} or {1}, put it onto the battlefield, then shuffle.'

In [303]:
re.findall(colorless_regex, urborg_text)

['{T}: Add {C}']

In [332]:
def determine_source(oracle_text, cmd_identity):
    
    if re.findall(fetch_regex, oracle_text):
        color = cmd_identity
    elif re.findall(wubrg_regex, oracle_text):
        color = cmd_identity
    elif re.findall(slow_fetch_regex, oracle_text):
        color = cmd_identity
    elif re.findall(urborg_regex, oracle_text):
        color = ["B"]
    elif re.findall(colorless_regex, oracle_text):
        color = ["C"]
    elif re.findall(pool_regex, oracle_text):
        color = cmd_identity
    else:
        color = "Undefined"
    
    return color
        

In [152]:
determine_source(fetch_text, deck_identity)

['B', 'R']